In [ ]:
%matplotlib auto
import numpy as np 
import matplotlib.pyplot as plt

In [2]:

def c_decode(c_str):
    c_coeff = []
    for i in c_str:
        if i == "2":
            c_coeff.append(-1)
        if i == "0":
            c_coeff.append(0)
        if i == "1":
            c_coeff.append(1)
    return c_coeff

# compute prob of tamplate origin
def prob(n, cov_mat, N):
    det = 0
    if N == 1:
        det = cov_mat[0]
    else:
        det = np.linalg.det(cov_mat)
    cov_mat_inv = np.linalg.pinv(cov_mat)
    return 1/np.sqrt(det * pow(2*np.pi,N))*np.exp(-0.5*np.dot(np.dot(n.T,cov_mat_inv), n))

# public info
tag = ["(-1,-1)","(-1,0)","(-1,1)","(0,-1)","(0,0)","(0,1)","(1,-1)","(1,0)","(1,1)"]
ave_len = 305
st = 21

In [3]:
# read test data from s

# coeffs of test data
c_lst_test = []
for line in open("test_trc/challenge_c.txt"):
    c_lst_test.append(c_decode(line[:-1]))

# traces of test data
mu = 0
sigma = 0.5
test_trc_lst = []
for q in range(0, len(c_lst_test)):
    filename = f"test_trc/test_trc{q+1}.trc"
    trc = np.loadtxt(filename, dtype="float32")
    trc += np.random.normal(mu, sigma, trc.shape)
    test_trc_lst.append(trc)

In [4]:
# load data (coeffs and traces) of load a & b

# coeffs of load a & b
c_lst_ldr_ab = []
for line in open("ab_tamplate/challenge_c.txt"):
    c_lst_ldr_ab.append(line[:-1])

# traces of load a & b
profiled_lst_ldr_ab = []
for i in range(0, len(c_lst_ldr_ab)):
    profiled_lst_ldr_ab.append(np.loadtxt(f"ab_tamplate/trc{i}.trc", dtype='float32'))


In [5]:
# build tamplates for load a, a in {-1,0,1}

L_ldr_a = 1536 # number of sample for EACH tamplate
K_ldr_a = 3 #  number of tamplate
N_ldr_a = 3 # number of POI
P_ldr_a = [227,268,270] # POI

assert(N_ldr_a == len(P_ldr_a))

# assign trace blks into corresponding Sample list
Sample_ldr_a = [] # sample traces for {-1,0,1}

for i in range(0, K_ldr_a):
    Sample_ldr_a.append([])

for j in range(0, len(c_lst_ldr_ab)):
   c_ans = c_decode(c_lst_ldr_ab[j])
   for i in range(0, 128):
        Sample_ldr_a[c_ans[i]+1].append(profiled_lst_ldr_ab[j][st+i*ave_len:st+(i+1)*ave_len])

for k in range(0, K_ldr_a):
    assert(len(Sample_ldr_a[k]) == L_ldr_a)

M_ldr_a = np.zeros([K_ldr_a, ave_len]) # average trace of each tamplate
# compute average trace Mi
for k in range(0, K_ldr_a):
    M_ldr_a[k] = np.mean(Sample_ldr_a[k], axis=0)

# compute noise vector for all L sample traces on POI j
noise_vec_ldr_a = np.zeros([K_ldr_a, L_ldr_a, N_ldr_a])
for k in range(0, K_ldr_a):
    for i in range(0, L_ldr_a):
        for j in range(0, N_ldr_a):
            noise_vec_ldr_a[k][i][j] = np.random.normal(0, 0.01) + Sample_ldr_a[k][i][P_ldr_a[j]] - M_ldr_a[k][P_ldr_a[j]]

# compute cov matrix of corresponding noise vector
cov_mat_ldr_a = np.zeros([K_ldr_a, N_ldr_a, N_ldr_a])
for k in range(0, K_ldr_a):
    cov_mat_ldr_a[k] = np.cov(noise_vec_ldr_a[k], rowvar=False)

# print(cov_mat_ldr_a.shape)


In [6]:
# test tamplate of load a
ave_success_rate = 0
for q in range(0, len(c_lst_test)):
    trc_blk= []
    for i in range(0, 128):
        trc_blk.append(test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len])

    assert(len(trc_blk)==128)

    poss_value_lst = []
    for block in trc_blk:
        poss_value_tmp = []
        nv = np.zeros([K_ldr_a, N_ldr_a])
        for k in range(0, K_ldr_a):
            for i in range(0, N_ldr_a):
                nv[k][i] = block[P_ldr_a[i]] - M_ldr_a[k][P_ldr_a[i]]
            poss_value_tmp.append(prob(nv[k], cov_mat_ldr_a[k], N_ldr_a))
        poss_value_lst.append(poss_value_tmp)

    poss_res_lst = []
    for poss_value_tmp in poss_value_lst:
        poss_res_tmp = [-1, 0, 1]
        zip_poss = zip(poss_res_tmp, poss_value_tmp)
        sorted_zip = sorted(zip_poss, key=lambda x:x[1], reverse=True)
        sorted_poss_res_tmp, sorted_poss_value = zip(*sorted_zip)
        poss_res_lst.append(sorted_poss_res_tmp[0])

    success_rate = 0
    for i in range(0, 128):
        if poss_res_lst[i] == c_lst_test[q][i]:
            success_rate += 1
    # print(f"trace {q+1}: success rate =",success_rate/128)
    ave_success_rate += success_rate
print("ave success rate = ", ave_success_rate/(128*len(c_lst_test)))

ave success rate =  1.0


In [7]:
# build tamplate for load b

L_ldr_b = 1536 # sample number
K_ldr_b = 3 # mode number
N_ldr_b = 3 # number of POI
P_ldr_b = [51,163,183] # POI

# assign trace blks into corresponding Sample list
Sample_ldr_b = [] # according array for sample traces

for i in range(0, K_ldr_b):
    Sample_ldr_b.append([])

for j in range(0, len(c_lst_ldr_ab)):
   c_ans = c_decode(c_lst_ldr_ab[j])[128:]
   for i in range(0, 128):
        Sample_ldr_b[c_ans[i]+1].append(profiled_lst_ldr_ab[j][st+i*ave_len:st+(i+1)*ave_len])

for k in range(0, K_ldr_b):
    assert(len(Sample_ldr_b[k])==L_ldr_b)

# compute average trace Mi
M_ldr_b = np.zeros([K_ldr_b, ave_len])
for k in range(0,K_ldr_b):
    M_ldr_b[k] = np.mean(Sample_ldr_b[k], axis=0)

# compute noise vector for all L sample traces on POI j
noise_vec_ldr_b = np.zeros([K_ldr_b, L_ldr_b, N_ldr_b])
for k in range(0,K_ldr_b):
    for i in range(0, L_ldr_b):
        for j in range(0, N_ldr_b):
            noise_vec_ldr_b[k][i][j] = np.random.normal(0, 0.5)+ Sample_ldr_b[k][i][P_ldr_b[j]] - M_ldr_b[k][P_ldr_b[j]]
# compute cov matrix of corresponding noise vector
cov_mat_ldr_b = np.zeros([K_ldr_b,N_ldr_b,N_ldr_b])
for k in range(0,K_ldr_b):
    cov_mat_ldr_b[k] = np.cov(noise_vec_ldr_b[k], rowvar=False)

# print(cov_mat_ldr_a.shape)

In [8]:
# test tamplate of load b
ave_success_rate = 0
for q in range(0, len(c_lst_test)):
    trc_blk= []
    for i in range(0,128):
        trc_blk.append(test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len])

    assert(len(trc_blk)==128)

    poss_value_list = []
    for block in trc_blk:
        poss_value_tmp = []
        nv = np.zeros([K_ldr_b, N_ldr_b])
        for k in range(0, K_ldr_b):
            for i in range(0, N_ldr_b):
                nv[k][i] = block[P_ldr_b[i]]-M_ldr_b[k][P_ldr_b[i]]
            poss_value_tmp.append(prob(nv[k], cov_mat_ldr_b[k], N_ldr_b))
        poss_value_list.append(poss_value_tmp)

    poss_res_list = []
    for poss_value_tmp in poss_value_list:
        poss_res_tmp = [-1, 0, 1]
        zip_poss = zip(poss_res_tmp,poss_value_tmp)
        sorted_zip = sorted(zip_poss, key=lambda x:x[1], reverse=True)
        sorted_poss_res_tmp, sorted_poss_value = zip(*sorted_zip)
        poss_res_list.append(sorted_poss_res_tmp[0])

    success_rate = 0
    for i in range(0, 128):
        if poss_res_list[i] == c_lst_test[q][i+128]:
            success_rate+=1
    # print(f"trace {q+1}: success rate =",success_rate/128)
    ave_success_rate += success_rate
print("ave success rate = ", ave_success_rate/(128*len(c_lst_test)))

ave success rate =  1.0


In [9]:
# build tamplate for store b
c_lst_str_b = []
for line in open("hwb_tamplate/challenge_c.txt"):
    c_lst_str_b.append(line[1:-3])

# hwb trc of profiled c
profiled_lst_str_b = []
for i in range(0, len(c_lst_str_b)):
    profiled_lst_str_b.append(np.loadtxt(f"hwb_tamplate/hwb{i+1}.trc", dtype='float32'))

hwb_set = [0,1,6,13,14,18,19,32]
hw_b = [[5,7,3], [6,0,4], [6,1,2]]
L_str_b = 1536 # sample number
K_str_b = 8 # number of tamplates
N_str_b = 1 # number of POI
P_str_b = [250] # POI

# assign trace blks into corresponding Sample list
Sample_str_b = [] # according array for sample traces
for i in range(0, K_str_b):
    Sample_str_b.append([])

for j in range(0, len(c_lst_str_b)):
   c_ans = c_decode(c_lst_str_b[j])
   for i in range(0, 128):
        Sample_str_b[hw_b[c_ans[i]+1][c_ans[i+128]+1]].append(profiled_lst_str_b[j][st+i*ave_len:st+(i+1)*ave_len])  

for k in range(0, K_str_b):
    assert(len(Sample_str_b[k]) == L_str_b)

M_str_b = np.zeros([K_str_b, ave_len])
# compute average trace Mi
for k in range(0, K_str_b):
    M_str_b[k] = np.mean(Sample_str_b[k], axis=0,dtype=np.float64)

# compute noise vector for all L sample traces on POI j
noise_vec_str_b = np.zeros([K_str_b, L_str_b, N_str_b])
for k in range(0, K_str_b):
    for i in range(0, L_str_b):
        for j in range(0, N_str_b):
            noise_vec_str_b[k][i][j] = np.random.normal(0, 0.1) + Sample_str_b[k][i][P_str_b[j]] - M_str_b[k][P_str_b[j]]


# compute cov matrix of corresponding noise vector
cov_mat_str_b = np.zeros([K_str_b, N_str_b, N_str_b])
for k in range(0, K_str_b):
    cov_mat_str_b[k] = np.cov(noise_vec_str_b[k], rowvar=False)

# print(cov_mat_ldr_a.shape)

In [10]:
# test tamplate of store b
ave_HW_1 = 0
ave_HW_5 = 0
HW1_num = np.zeros(K_str_b)
HW5_num = np.zeros(K_str_b)
cnt_num = np.zeros(K_str_b)
for q in range(0, len(c_lst_test)):
    trc_blk= []
    for i in range(0,128):
        trc_blk.append(test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len])

    assert(len(trc_blk)==128)

    poss_value_list = []
    for block in trc_blk:
        poss_value_tmp = []
        nv = np.zeros([K_str_b, N_str_b])
        for k in range(0, K_str_b):
            for i in range(0, N_str_b):
                nv[k][i] = block[P_str_b[i]]-M_str_b[k][P_str_b[i]]
            poss_value_tmp.append(prob(nv[k], cov_mat_str_b[k], N_str_b))
        poss_value_list.append(poss_value_tmp)

    poss_res_list = []
    for poss_value_tmp in poss_value_list:
        poss_res_tmp = [0,1,6,13,14,18,19,32]
        zip_poss = zip(poss_res_tmp,poss_value_tmp)
        sorted_zip = sorted(zip_poss, key=lambda x:x[1], reverse=True)
        sorted_poss_res_tmp, sorted_poss_value = zip(*sorted_zip)
        poss_res_list.append(sorted_poss_res_tmp[0:5])

    HW_1 = 0
    HW_5 = 0
    ct_ans = c_lst_test[q]
    for i in range(0, 128):
        hw = hwb_set[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]
        cnt_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        if poss_res_list[i][0] == hw:
            HW_1 += 1
            HW1_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        if hw in poss_res_list[i]:
            HW_5 += 1
            HW5_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        

        '''
        if hw!=poss_res_list[i][0]:
            print("poss = ", poss_res_list[i], "ans = ", hw)
        '''
    # print(f"trace {q}: HW 1 rate =",HW_1/128)
    # print(f"trace {q}: HW 5 rate =",HW_5/128)
    ave_HW_1 += HW_1
    ave_HW_5 += HW_5
print("ave HW 1 = ", ave_HW_1/(128*len(c_lst_test)))
print("ave HW 5 = ", ave_HW_5/(128*len(c_lst_test)))
for k in range(0, K_str_b):
    print(f"tamplate{hwb_set[k]}:")
    print("ave HW1 = ", HW1_num[k]/cnt_num[k])
    print("ave HW5 = ", HW5_num[k]/cnt_num[k])

ave HW 1 =  0.999267578125
ave HW 5 =  1.0
tamplate0:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate1:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate6:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate13:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate14:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate18:
ave HW1 =  0.9318181818181818
ave HW5 =  1.0
tamplate19:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate32:
ave HW1 =  1.0
ave HW5 =  1.0


In [11]:
# build tamplate for store a
c_lst_str_a = []
for line in open("hwa_tamplate/challenge_c.txt"):
    c_lst_str_a.append(line[1:-3])


# hwa trc of profiled c
profiled_lst_str_a = []
for i in range(0, len(c_lst_str_a)):
    profiled_lst_str_a.append(np.loadtxt(f"hwa_tamplate/hwa{i+1}.trc", dtype='float32'))

hwa_set = [0,1,6,13,14,18,191,192,32]
hw_a = [[3,8,5], [4,0,6], [2,1,7]]
L_str_a = 1536 # sample number
K_str_a = 9 # number of tamplates
N_str_a = 1 # number of POI
P_str_a = [292] # POI

# assign trace blks into corresponding Sample list
Sample_str_a = [] # according array for sample traces
for i in range(0, K_str_a):
    Sample_str_a.append([])

for j in range(0, len(c_lst_str_a)):
   c_ans = c_decode(c_lst_str_a[j])
   for i in range(0, 128):
        Sample_str_a[hw_a[c_ans[i]+1][c_ans[i+128]+1]].append(profiled_lst_str_a[j][st+i*ave_len:st+(i+1)*ave_len])

for k in range(0, K_str_a):
    # assert(len(Sample_str_a[k]) == L_str_a)
    print(len(Sample_str_a[k]))

# compute average trace Mi
M_str_a = np.zeros([K_str_a, ave_len])
for k in range(0, K_str_a):
    M_str_a[k] = np.mean(Sample_str_a[k], axis=0)

# compute noise vector for all L sample traces on POI j
noise_vec_str_a = np.zeros([K_str_a, L_str_a, N_str_a])
for k in range(0, K_str_a):
        for i in range(0, L_str_a):
            for j in range(0, N_str_a):
                noise_vec_str_a[k][i][j] = np.random.normal(0, 0.1) + Sample_str_a[k][i][P_str_a[j]] - M_str_a[k][P_str_a[j]]

# compute cov matrix of corresponding noise vector
cov_mat_str_a = np.zeros([K_str_a, N_str_a, N_str_a])
for k in range(0, K_str_a):
    cov_mat_str_a[k] = np.cov(noise_vec_str_a[k], rowvar=False)

# print(cov_mat_ldr_a.shape)

1536
1536
1536
1536
1536
1536
1536
1536
1536


In [12]:
# test tamplate of store a
ave_HW_1 = 0
ave_HW_5 = 0
HW1_num = np.zeros(K_str_b)
HW5_num = np.zeros(K_str_b)
cnt_num = np.zeros(K_str_b)
for q in range(0, len(c_lst_test)):
    trc_blk = []
    for i in range(0, 128):
        trc_blk.append(test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len])

    assert(len(trc_blk)==128)

    poss_value_list = []
    for block in trc_blk:
        poss_value_tmp = []
        nv = np.zeros([K_str_a, N_str_a])
        for k in range(0, K_str_a):
            for i in range(0, N_str_a):
                nv[k][i] = block[P_str_a[i]]-M_str_a[k][P_str_a[i]]
            poss_value_tmp.append(prob(nv[k], cov_mat_str_a[k], N_str_a))
        poss_value_list.append(poss_value_tmp)

    poss_res_list = []
    prob_res_lst = []
    for poss_value_tmp in poss_value_list:
        poss_res_tmp = [0,1,6,13,14,18,191,192,32]
        zip_poss = zip(poss_res_tmp,poss_value_tmp)
        sorted_zip = sorted(zip_poss, key=lambda x:x[1], reverse=True)
        sorted_poss_res_tmp, sorted_poss_value = zip(*sorted_zip)
        poss_res_list.append(sorted_poss_res_tmp[0:5])
        prob_res_lst.append(sorted_poss_value[0:5])
    HW_1 = 0
    HW_5 = 0
    ct_ans = c_lst_test[q]
    for i in range(0, 128):
        hw = hwa_set[hw_a[ct_ans[i]+1][ct_ans[i+128]+1]]
        cnt_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        if poss_res_list[i][0] == hw:
            HW_1 += 1
            HW1_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        #elif poss_res_list[i][0] == 19 and prob_res_lst[i][1]!=0 and poss_res_list[i][1]==hw:
        #   HW_1 += 1
        if hw in poss_res_list[i]:
            HW_5 += 1
            HW5_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        '''
        if poss_res_list[i][0] != hw:
            for k in range(0, K_str_a):
                plt.plot(M_str_a[k], label = hwa_set[k])
                print(M_str_a[k][292])
            plt.plot(trc_blk[i], label ="test")
            print(trc_blk[i][292])
            plt.legend()
            plt.show()
            print("poss = ", poss_res_list[i], "ans = ", hw)
            print("prob = ", prob_res_lst[i])
        '''
    # print(f"trace {q}: HW 1 rate =",HW_1/128)
    # print(f"trace {q}: HW 5 rate =",HW_5/128)
    ave_HW_1 += HW_1
    ave_HW_5 += HW_5
print("ave HW 1 = ", ave_HW_1/(128*len(c_lst_test)))
print("ave HW 5 = ", ave_HW_5/(128*len(c_lst_test)))  
for k in range(0, K_str_b):
    print(f"tamplate{hwb_set[k]}:")
    print("ave HW1 = ", HW1_num[k]/cnt_num[k])
    print("ave HW5 = ", HW5_num[k]/cnt_num[k])

ave HW 1 =  1.0
ave HW 5 =  1.0
tamplate0:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate1:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate6:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate13:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate14:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate18:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate19:
ave HW1 =  1.0
ave HW5 =  1.0
tamplate32:
ave HW1 =  1.0
ave HW5 =  1.0
